In [1]:
import regex as re
import pandas as pd
import glob
import tabula
import pymupdf4llm
import requests
import fitz  # PyMuPDF

In [2]:
!pip install pymupdf4llm

In [3]:
base_url = "https://adamswebsearch2.nrc.gov/webSearch2/main.jsp?AccessionNumber="

In [4]:
inspection_ids = ["ML23129A788", "ML23067A022", "ML23128A148", "ML23124A225", "ML23082A034", "ML23067A024", "ML23115A000", "ML23048A062", "ML23122A221", "ML23128A351", "ML23089A119", "ML23108A332", "ML23124A386", "ML23088A167", "ML23102A139", "ML23114A173", "ML23059A274", "ML23114A383", "ML23124A007", "ML23123A414", "ML23054A359", "ML23122A076", "ML23089A166", "ML23080A298", "ML23102A149", "ML23023A058", "ML23060A158", "ML23101A121", "ML23074A227", "ML23030B899", "ML23122A100", "ML23116A173", "ML23025A407", "ML23072A423", "ML23068A069", "ML23121A139", "ML23023A100", "ML23110A634", "ML23116A242", "ML23068A150", "ML23124A273", "ML23089A399", "ML23089A355", "ML23102A039", "ML23122A168", "ML23044A155", "ML23075A135", "ML23095A085", "ML23123A168", "ML23116A012", "ML23062A449", "ML23110A800", "ML23095A332", "ML23089A021", "ML23073A029", "ML23129A251", "ML23111A294", "ML23118A051", "ML23116A003", "ML23130A218", "ML23038A012", "ML23118A027", "ML23118A014", "ML23125A125", "ML23026A048", "ML23122A162", "ML23124A108", "ML23125A005", "ML23118A017", "ML23104A043", "ML23130A412", "ML23088A360", "ML23053A079", "ML23124A062", "ML23103A264", "ML23086A209", "ML23069A019", "ML23128A135", "ML23122A272", "ML23053A049", "ML23053A049", "ML23121A104", "ML23130A172", "ML23128A101", "ML23102A294", "ML23116A235", "ML23121A172", "ML23075A146", "ML23123A065", "ML23066A013", "ML23122A217", "ML23052A056", "ML23129A193", "ML23103A395", "ML23044A099", "ML23102A113", "ML23033A155", "ML23121A321", "ML23074A192", "ML23121A105", "ML23110A664", "ML23032A427", "ML23075A170", "ML23114A094", "ML23131A351", "ML23123A122", "ML23132A143", "ML23132A307", "ML23047A033", "ML23114A048", "ML23121A320", "ML23129A022", "ML23107A171", "ML23025A384", "ML23110A762", "ML23068A133", "ML23123A054", "ML23032A374", "ML23102A200", "ML23111A066"]

In [5]:
inspection_id_urls_list = []

for inspection_id in inspection_ids:
    updated_id_links = base_url + inspection_id
    inspection_id_urls_list.append(updated_id_links)

inspection_id_urls_list

['https://adamswebsearch2.nrc.gov/webSearch2/main.jsp?AccessionNumber=ML23129A788',
 'https://adamswebsearch2.nrc.gov/webSearch2/main.jsp?AccessionNumber=ML23067A022',
 'https://adamswebsearch2.nrc.gov/webSearch2/main.jsp?AccessionNumber=ML23128A148',
 'https://adamswebsearch2.nrc.gov/webSearch2/main.jsp?AccessionNumber=ML23124A225',
 'https://adamswebsearch2.nrc.gov/webSearch2/main.jsp?AccessionNumber=ML23082A034',
 'https://adamswebsearch2.nrc.gov/webSearch2/main.jsp?AccessionNumber=ML23067A024',
 'https://adamswebsearch2.nrc.gov/webSearch2/main.jsp?AccessionNumber=ML23115A000',
 'https://adamswebsearch2.nrc.gov/webSearch2/main.jsp?AccessionNumber=ML23048A062',
 'https://adamswebsearch2.nrc.gov/webSearch2/main.jsp?AccessionNumber=ML23122A221',
 'https://adamswebsearch2.nrc.gov/webSearch2/main.jsp?AccessionNumber=ML23128A351',
 'https://adamswebsearch2.nrc.gov/webSearch2/main.jsp?AccessionNumber=ML23089A119',
 'https://adamswebsearch2.nrc.gov/webSearch2/main.jsp?AccessionNumber=ML2310

In [6]:
inspection_ids_pdfs = [
    "ML23129A788.pdf", "ML23067A022.pdf", "ML23128A148.pdf", "ML23124A225.pdf", "ML23082A034.pdf", "ML23067A024.pdf", 
    "ML23115A000.pdf", "ML23048A062.pdf", "ML23122A221.pdf", "ML23128A351.pdf", "ML23089A119.pdf", "ML23108A332.pdf", 
    "ML23124A386.pdf", "ML23088A167.pdf", "ML23102A139.pdf", "ML23114A173.pdf", "ML23059A274.pdf", "ML23114A383.pdf", 
    "ML23124A007.pdf", "ML23123A414.pdf", "ML23054A359.pdf", "ML23122A076.pdf", "ML23089A166.pdf", "ML23080A298.pdf", 
    "ML23102A149.pdf", "ML23023A058.pdf", "ML23060A158.pdf", "ML23101A121.pdf", "ML23074A227.pdf", "ML23030B899.pdf", 
    "ML23122A100.pdf", "ML23116A173.pdf", "ML23025A407.pdf", "ML23072A423.pdf", "ML23068A069.pdf", "ML23121A139.pdf", 
    "ML23023A100.pdf", "ML23110A634.pdf", "ML23116A242.pdf", "ML23068A150.pdf", "ML23124A273.pdf", "ML23089A399.pdf", 
    "ML23089A355.pdf", "ML23102A039.pdf", "ML23122A168.pdf", "ML23044A155.pdf", "ML23075A135.pdf", "ML23095A085.pdf", 
    "ML23123A168.pdf", "ML23116A012.pdf", "ML23062A449.pdf", "ML23110A800.pdf", "ML23095A332.pdf", "ML23089A021.pdf", 
    "ML23073A029.pdf", "ML23129A251.pdf", "ML23111A294.pdf", "ML23118A051.pdf", "ML23116A003.pdf", "ML23130A218.pdf", 
    "ML23038A012.pdf", "ML23118A027.pdf", "ML23118A014.pdf", "ML23125A125.pdf", "ML23026A048.pdf", "ML23122A162.pdf", 
    "ML23124A108.pdf", "ML23125A005.pdf", "ML23118A017.pdf", "ML23104A043.pdf", "ML23130A412.pdf", "ML23088A360.pdf", 
    "ML23053A079.pdf", "ML23124A062.pdf", "ML23103A264.pdf", "ML23086A209.pdf", "ML23069A019.pdf", "ML23128A135.pdf", 
    "ML23122A272.pdf", "ML23053A049.pdf", "ML23053A049.pdf", "ML23121A104.pdf", "ML23130A172.pdf", "ML23128A101.pdf", 
    "ML23102A294.pdf", "ML23116A235.pdf", "ML23121A172.pdf", "ML23075A146.pdf", "ML23123A065.pdf", "ML23066A013.pdf", 
    "ML23122A217.pdf", "ML23052A056.pdf", "ML23129A193.pdf", "ML23103A395.pdf", "ML23044A099.pdf", "ML23102A113.pdf", 
    "ML23033A155.pdf", "ML23121A321.pdf", "ML23074A192.pdf", "ML23121A105.pdf", "ML23110A664.pdf", "ML23032A427.pdf", 
    "ML23075A170.pdf", "ML23114A094.pdf", "ML23131A351.pdf", "ML23123A122.pdf", "ML23132A143.pdf", "ML23132A307.pdf", 
    "ML23047A033.pdf", "ML23114A048.pdf", "ML23121A320.pdf", "ML23129A022.pdf", "ML23107A171.pdf", "ML23025A384.pdf", 
    "ML23110A762.pdf", "ML23068A133.pdf", "ML23123A054.pdf", "ML23032A374.pdf", "ML23102A200.pdf", "ML23111A066.pdf"
]


In [7]:
def extract_text_from_pdf(pdf_path): ##retrieving text from pdfs 
    doc = pymupdf4llm.open(pdf_path)
    
    text = ""
    for page_num in range(doc.page_count):
        page = doc.load_page(page_num)
        text += page.get_text("text")
    
    return text

In [8]:
def apply_regex_to_text(text, pattern):
    return re.findall(pattern, text)

In [9]:
inspectdate_pat = r"(?<=completed an inspection\s+at.*?)(\w+\s+\d{1,2},\s+\d{4})" ##this and following lines establishing regex for finding necessary scrapable data
plantname_pat = r"SUBJECT:\s(\w+(\s\w+)*)"
violation_pat = r"(?:violation|apparent violation.*?)(?:\(\w+\))?"

In [10]:
df = pd.DataFrame(columns=['Plant Name', 'Inspection Date', 'Violation?'])

In [11]:
pdf_files = glob.glob("*.pdf")

In [12]:
pip install PyMuPDF


Note: you may need to restart the kernel to use updated packages.


In [13]:
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    
    text = ""
    for page_num in range(doc.page_count):
        page = doc.load_page(page_num)
        text += page.get_text("text")
    
    return text ##function used 

data = []

pdf_files = glob.glob("*.pdf") ##finding appropriate files

for pdf_file in pdf_files:
    text = extract_text_from_pdf(pdf_file)
    
    plant_name = re.findall(plantname_pat, text) ##this and next 2 lines are regex application
    date = re.findall(inspectdate_pat, text)
    violation = re.findall(violation_pat, text)
    
    if plant_name and date and violation: ##appending to dataframe if something is found
        data.append({
            'Plant Name': plant_name[0],
            'Inspection Date': date[0],
            'Violation?': violation[0]
        })
    else: ##appending to dataframe if nothing is found
        data.append({
            'Plant Name': "n/a",
            'Inspection Date': "n/a",
            'Violation?': "n/a"
        })

df = pd.DataFrame(data)

print(df)


  Plant Name Inspection Date Violation?
0        n/a             n/a        n/a
1        n/a             n/a        n/a
2        n/a             n/a        n/a
3        n/a             n/a        n/a


In [14]:
df.to_csv('nucplantviolations.csv', index=False)

In [15]:
!pip install tabular-2-text

In [16]:
!pip install fuzzy-context-finder
from fuzzy_context_finder import keyword_context_finder

In [17]:
pip install mangoCR --upgrade

Note: you may need to restart the kernel to use updated packages.


In [18]:
target_pdfs = glob.glob("pdfs/ML*.pdf")
target_pdfs ##establishing pdfs to check


['pdfs/ML23030B899.pdf',
 'pdfs/ML23088A167.pdf',
 'pdfs/ML23047A033.pdf',
 'pdfs/ML23121A104.pdf',
 'pdfs/ML23110A634.pdf',
 'pdfs/ML23121A139.pdf',
 'pdfs/ML23121A105.pdf',
 'pdfs/ML23124A108.pdf',
 'pdfs/ML23114A173.pdf',
 'pdfs/ML23125A125.pdf',
 'pdfs/ML23048A062.pdf',
 'pdfs/ML23115A000.pdf',
 'pdfs/ML23025A384.pdf',
 'pdfs/ML23068A150.pdf',
 'pdfs/ML23102A294.pdf',
 'pdfs/ML23023A100.pdf',
 'pdfs/ML23038A012.pdf',
 'pdfs/ML23059A274.pdf',
 'pdfs/ML23128A101.pdf',
 'pdfs/ML23122A162.pdf',
 'pdfs/ML23023A058.pdf',
 'pdfs/ML23069A019.pdf',
 'pdfs/ML23089A399.pdf',
 'pdfs/ML23110A800.pdf',
 'pdfs/ML23118A027.pdf',
 'pdfs/ML23088A360.pdf',
 'pdfs/ML23123A414.pdf',
 'pdfs/ML23122A217.pdf',
 'pdfs/ML23089A166.pdf',
 'pdfs/ML23107A171.pdf',
 'pdfs/ML23066A013.pdf',
 'pdfs/ML23072A423.pdf',
 'pdfs/ML23095A085.pdf',
 'pdfs/ML23101A121.pdf',
 'pdfs/ML23095A332.pdf',
 'pdfs/ML23122A272.pdf',
 'pdfs/ML23068A133.pdf',
 'pdfs/ML23132A307.pdf',
 'pdfs/ML23116A012.pdf',
 'pdfs/ML23032A374.pdf',


In [21]:
keywords = [
    "violation",
    "violations",
    "enforcement",
    "action",
    "safety significance",
    "findings",
    "Red",
    "contest",
    "IV",
    "NCV",
    "Green",
    "NCVs",
    "severity",
    "denial",
    "failure"
]

## most common words associated with violations

In [22]:
from mangoCR import pdf2image_ocr_text
text = pdf2image_ocr_text(target_pdfs[-1]) ## checking one file
text

Processing PDF 1 of 1: ML23074A227
  - Processed page 1 of 6 in ML23074A227
  - Processed page 2 of 6 in ML23074A227
  - Processed page 3 of 6 in ML23074A227
  - Processed page 4 of 6 in ML23074A227
  - Processed page 5 of 6 in ML23074A227
  - Processed page 6 of 6 in ML23074A227
Finished processing ML23074A227

All PDFs have been processed.


'## ML23074A227_page_1\nUNITED STATES\nNUCLEAR REGULATORY COMMISSION\n\nREGION III\n2443 WARRENVILLE ROAD, SUITE 210\nLISLE, ILLINOIS 60532-4352\n\nMarch 16, 2023\n\nQ. Shane Lies\n\nSenior VP and Chief Nuclear Officer\nIndiana Michigan Power Company\nNuclear Generation Group\n\nOne Cook Place\n\nBridgman, MI 49106\n\nSUBJECT: | DONALD C. COOK NUCLEAR PLANT — SECURITY BASELINE INSPECTION\nREPORT 05000315/2023401 AND 05000316/2023401\n\nDear Q. Shane Lies:\n\nOn February 21, 2023, the U.S. Nuclear Regulatory Commission (NRC) completed an\ninspection at Donald C. Cook Nuclear Plant and discussed the results of this inspection with you\nand other members of your staff. The results of this inspection are documented in the enclosed\nreport.\n\nNo findings or violations of more than minor significance were identified during this inspection.\n\nThis letter, its enclosure, and your response (if any) will be made available for public inspection\nand copying at http:/Awww.nrc.gov/reading-rm/adam

In [23]:
df = keyword_context_finder(text, keywords, 'pdfs/ML23023A058.pdf', match_threshold=70  ) ##applying keywords to see how data works on one page
df

,File Name,Page Marker,Page Number,Matched Term,Original Term,Similarity Score,Search Term with 50 Words Context,Previous 250 Words (Including Term),Next 250 Words (Including Term)
0,pdfs/ML23023A058.pdf,ML23074A227_page_2,2,SECURITY,severity,75.000000,and Chief Nuclear Officer Indiana Michigan Pow...,UNITED STATES NUCLEAR REGULATORY COMMISSION RE...,SECURITY BASELINE INSPECTION REPORT 05000315/2...
1,pdfs/ML23023A058.pdf,ML23074A227_page_2,2,findings,findings,100.000000,the results of this inspection with you and ot...,UNITED STATES NUCLEAR REGULATORY COMMISSION RE...,findings or violations of more than minor sign...
2,pdfs/ML23023A058.pdf,ML23074A227_page_2,2,violations,violation,94.736842,of this inspection with you and other members ...,UNITED STATES NUCLEAR REGULATORY COMMISSION RE...,violations of more than minor significance wer...
3,pdfs/ML23023A058.pdf,ML23074A227_page_2,2,violations,violations,100.000000,of this inspection with you and other members ...,UNITED STATES NUCLEAR REGULATORY COMMISSION RE...,violations of more than minor significance wer...
4,pdfs/ML23023A058.pdf,ML23074A227_page_2,2,significance,safety significance,77.419355,and other members of your staff. The results o...,UNITED STATES NUCLEAR REGULATORY COMMISSION RE...,significance were identified during this inspe...
5,pdfs/ML23023A058.pdf,ML23074A227_page_2,2,"Security,",severity,70.588235,"of the Code of Federal Regulations 2.390, “Pub...",UNITED STATES NUCLEAR REGULATORY COMMISSION RE...,"Security, Emergency Preparedness & Incident Re..."
6,pdfs/ML23023A058.pdf,ML23074A227_page_2,2,Security,severity,75.000000,"Withholding.” Sincerely, BE Signed by Orth, St...",UNITED STATES NUCLEAR REGULATORY COMMISSION RE...,Security Docket Nos. 05000315 and 05000316 Lic...
7,pdfs/ML23023A058.pdf,ML23074A227_page_3,3,SECURITY,severity,75.000000,Letter to Q. S. Lies from S. Orth dated March ...,Letter to Q. S. Lies from S. Orth dated March ...,SECURITY BASELINE INSPECTION REPORT 05000315/2...
8,pdfs/ML23023A058.pdf,ML23074A227_page_3,3,Jared,Red,75.000000,RidsNrrPMDCCook Resource RidsNrrDrolrib Resour...,Letter to Q. S. Lies from S. Orth dated March ...,Jared Justice Jannette Worosilo Ray Kellar Nat...
9,pdfs/ML23023A058.pdf,ML23074A227_page_4,4,Security,severity,75.000000,and 05000316/2023401 I-2023-401-0058 Indiana M...,Docket Numbers: License Numbers: Report Number...,"Security Inspector Steven K. Orth, Chief Secur..."


In [24]:
from tabular_2_text import tab2text

In [25]:
tab2text(df, "md_1_file.txt") ## running tab2text on file to check for efficiency

md_1_file.txt is in your local project folder.


'md_1_file.txt'

In [26]:
df_list = []
empty_list = []
for count, target_pdf in enumerate(target_pdfs, start=1):  # Use enumerate for progress tracking
    print(f"Processing file {count}/{len(target_pdfs)}: {target_pdf}")
    
    text = pdf2image_ocr_text(target_pdf)  # Extract text from the PDF
    
    try:
        # Append the DataFrame with results to df_list
        df_list.append(keyword_context_finder(text, keywords, target_pdf, match_threshold=76))
    except:
        # Handle files where no matches are found or other errors occur
        print(f"No words found in {target_pdf}")
        empty_list.append(target_pdf)
        
print("DONE PROCESSING!")

Processing file 1/107: pdfs/ML23030B899.pdf
Processing PDF 1 of 1: ML23030B899
  - Processed page 1 of 8 in ML23030B899
  - Processed page 2 of 8 in ML23030B899
  - Processed page 3 of 8 in ML23030B899
  - Processed page 4 of 8 in ML23030B899
  - Processed page 5 of 8 in ML23030B899
  - Processed page 6 of 8 in ML23030B899
  - Processed page 7 of 8 in ML23030B899
  - Processed page 8 of 8 in ML23030B899
Finished processing ML23030B899

All PDFs have been processed.
Processing file 2/107: pdfs/ML23088A167.pdf
Processing PDF 1 of 1: ML23088A167
  - Processed page 1 of 9 in ML23088A167
  - Processed page 2 of 9 in ML23088A167
  - Processed page 3 of 9 in ML23088A167
  - Processed page 4 of 9 in ML23088A167
  - Processed page 5 of 9 in ML23088A167
  - Processed page 6 of 9 in ML23088A167
  - Processed page 7 of 9 in ML23088A167
  - Processed page 8 of 9 in ML23088A167
  - Processed page 9 of 9 in ML23088A167
Finished processing ML23088A167

All PDFs have been processed.
Processing file 3/1

In [27]:
len(df_list) ## checking length for preliminary data eval

107

In [28]:
df = pd.concat(df_list, ignore_index = True) ## getting rid of index for clarity
df

,File Name,Page Marker,Page Number,Matched Term,Original Term,Similarity Score,Search Term with 50 Words Context,Previous 250 Words (Including Term),Next 250 Words (Including Term)
0,pdfs/ML23030B899.pdf,ML23030B899_page_2,2,findings,findings,100.000000,the results of this inspection with you and ot...,UNITED STATES NUCLEAR REGULATORY COMMISSION RE...,findings or violations of more than minor sign...
1,pdfs/ML23030B899.pdf,ML23030B899_page_2,2,violations,violation,94.736842,of this inspection with you and other members ...,UNITED STATES NUCLEAR REGULATORY COMMISSION RE...,violations of more than minor significance wer...
2,pdfs/ML23030B899.pdf,ML23030B899_page_2,2,violations,violations,100.000000,of this inspection with you and other members ...,UNITED STATES NUCLEAR REGULATORY COMMISSION RE...,violations of more than minor significance wer...
3,pdfs/ML23030B899.pdf,ML23030B899_page_2,2,significance,safety significance,77.419355,and other members of your staff. The results o...,UNITED STATES NUCLEAR REGULATORY COMMISSION RE...,significance were identified during this inspe...
4,pdfs/ML23030B899.pdf,ML23030B899_page_5,5,Findings,findings,100.000000,Reactor Oversight Process is the NRC’s program...,SUMMARY The U.S. Nuclear Regulatory Commission...,Findings and Violations No findings or violati...
...,...,...,...,...,...,...,...,...,...
11371,pdfs/ML23074A227.pdf,ML23074A227_page_5,5,findings,findings,100.000000,the NRC’s program for overseeing the safe oper...,SUMMARY The U.S. Nuclear Regulatory Commission...,findings or violations of more than minor sign...
11372,pdfs/ML23074A227.pdf,ML23074A227_page_5,5,violations,violation,94.736842,program for overseeing the safe operation of c...,SUMMARY The U.S. Nuclear Regulatory Commission...,violations of more than minor significance wer...
11373,pdfs/ML23074A227.pdf,ML23074A227_page_5,5,violations,violations,100.000000,program for overseeing the safe operation of c...,SUMMARY The U.S. Nuclear Regulatory Commission...,violations of more than minor significance wer...
11374,pdfs/ML23074A227.pdf,ML23074A227_page_5,5,significance,safety significance,77.419355,operation of commercial nuclear power reactors...,SUMMARY The U.S. Nuclear Regulatory Commission...,significance were identified. Additional Track...


In [29]:
df.to_csv("nucplanttextanalysis.csv", index = False) ## saving csv!!! yay